<a href="https://colab.research.google.com/github/cmiddleman/capstone3/blob/main/amazon_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
!pip install -q -U tensorflow-text

     |████████████████████████████████| 4.3 MB 24.8 MB/s 


In [10]:
!pip install -q -U tf-models-official

     |████████████████████████████████| 1.6 MB 19.2 MB/s 
     |████████████████████████████████| 352 kB 53.3 MB/s 
     |████████████████████████████████| 1.2 MB 52.9 MB/s 
     |████████████████████████████████| 99 kB 8.8 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 37.1 MB 110 kB/s 
     |████████████████████████████████| 211 kB 55.6 MB/s 
     |████████████████████████████████| 636 kB 33.9 MB/s 
     |████████████████████████████████| 679 kB 34.8 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 


In [11]:
!pip install -U tfds-nightly

     |████████████████████████████████| 4.0 MB 30.6 MB/s 


In [12]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text  # A dependency of the preprocessing model
import tensorflow_addons as tfa
from official.nlp import optimization
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import ast
import pickle
import joblib

tf.get_logger().setLevel('ERROR')

In [13]:
#automatic in colab I think?
os.environ["TFHUB_MODEL_LOAD_FORMAT"]="UNCOMPRESSED"

In [ ]:
import os

if os.environ['COLAB_TPU_ADDR']:
  cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
  tf.config.experimental_connect_to_cluster(cluster_resolver)
  tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
  strategy = tf.distribute.TPUStrategy(cluster_resolver)
  print('Using TPU')
elif tf.test.is_gpu_available():
  strategy = tf.distribute.MirroredStrategy()
  print('Using GPU')
else:
  raise ValueError('Running on CPU is not recommended.')

Using TPU


In [2]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [3]:
def make_bert_preprocess_model(sentence_features, seq_length=128):
  """Returns Model mapping string features to BERT inputs.

  Args:
    sentence_features: a list with the names of string-valued features.
    seq_length: an integer that defines the sequence length of BERT inputs.

  Returns:
    A Keras Model that can be called on a list or dict of string Tensors
    (with the order or names, resp., given by sentence_features) and
    returns a dict of tensors for input to BERT.
  """

  input_segments = [
      tf.keras.layers.Input(shape=(), dtype=tf.string, name=ft)
      for ft in sentence_features]

  # Tokenize the text to word pieces.
  bert_preprocess = hub.load(tfhub_handle_preprocess)
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  segments = [tokenizer(s) for s in input_segments]

  # Optional: Trim segments in a smart way to fit seq_length.
  # Simple cases (like this example) can skip this step and let
  # the next step apply a default truncation to approximately equal lengths.
  truncated_segments = segments

  # Pack inputs. The details (start/end token ids, dict of output tensors)
  # are model-dependent, so this gets loaded from the SavedModel.
  packer = hub.KerasLayer(bert_preprocess.bert_pack_inputs,
                          arguments=dict(seq_length=seq_length),
                          name='packer')
  model_inputs = packer(truncated_segments)
  return tf.keras.Model(input_segments, model_inputs)

In [14]:
bert_preprocessing_model = make_bert_preprocess_model(['text'])

In [15]:
drive_path = '/content/drive/MyDrive/amazon_reviews/'

In [ ]:
train_df = pd.read_csv(drive_path + 'data/train.csv')
test_df = pd.read_csv(drive_path + 'data/test.csv')

In [ ]:
smol_train_df = train_df[:9000].copy()
smol_test_df = test_df[:1000].copy()

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

def load_dataset_from_pd(df, bert_preprocess_model, batch_size=32):
    ds = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(df['text'].values, tf.string),
            tf.cast(df['rating'].values, tf.int32)
        )
    )
)
    ds = ds.batch(batch_size)
    ds = ds.map(lambda text,rating: (bert_preprocess_model(text),rating))
    ds = ds.cache().prefetch(buffer_size=AUTOTUNE)
    return ds, len(df)

In [ ]:
def save_bert_model(model, preprocess_model, path):

    preprocess_inputs = preprocess_model.inputs
    bert_encoder_inputs = preprocess_model(preprocess_inputs)
    bert_outputs = model(bert_encoder_inputs)
    model_for_export = tf.keras.Model(preprocess_inputs, bert_outputs)

    print('Saving', path)

    # Save everything on the Colab host (even the variables from TPU memory)
    save_options = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
    model_for_export.save(path, include_optimizer=False,
                        options=save_options)

In [16]:
def build_classifier_model(num_classes):

  class Classifier(tf.keras.Model):
    def __init__(self, num_classes):
      super(Classifier, self).__init__(name="prediction")
      self.encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True)
      self.dropout = tf.keras.layers.Dropout(0.1)
      self.dense = tf.keras.layers.Dense(num_classes)

    def call(self, preprocessed_text):
      encoder_outputs = self.encoder(preprocessed_text)
      pooled_output = encoder_outputs["pooled_output"]
      x = self.dropout(pooled_output)
      x = self.dense(x)
      return x

  model = Classifier(num_classes)
  return model

In [ ]:
epochs = 3
batch_size = 32
init_lr = 1e-5



with strategy.scope():

  # metric have to be created inside the strategy scope

    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metrics = tf.keras.metrics.SparseCategoricalAccuracy(
        'accuracy', dtype=tf.float32)

    train_dataset, train_data_size = load_dataset_from_pd(smol_train_df,bert_preprocessing_model, batch_size)
    steps_per_epoch = train_data_size // batch_size
    num_train_steps = steps_per_epoch * epochs
    num_warmup_steps = num_train_steps // 10

    validation_dataset, validation_data_size = load_dataset_from_pd(smol_test_df,bert_preprocessing_model, batch_size)
    validation_steps = validation_data_size // batch_size

    classifier_model = build_classifier_model(2)

    optimizer = optimization.create_optimizer(
        init_lr=init_lr,
        num_train_steps=num_train_steps,
        num_warmup_steps=num_warmup_steps,
        optimizer_type='adamw')

    classifier_model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])

# model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
# filepath=drive_path + 'models/callbacktest2',
# save_weights_only=False,
# monitor='val_accuracy',
# mode='auto',
# save_best_only=False)


for i in range(epochs):
    classifier_model.fit(
        x=train_dataset,
        validation_data=validation_dataset,
        steps_per_epoch=steps_per_epoch,
        epochs=1,
        validation_steps=validation_steps,
        verbose=1)

        #save_bert_model(classifier_model, bert_preprocessing_model, drive_path + 'models/amzn_model' + str(i+1))
    

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("AdamWeightDecay/gradients/StatefulPartitionedCall:1", shape=(None,), dtype=int32), values=Tensor("clip_by_global_norm/clip_by_global_norm/_0:0", dtype=float32), dense_shape=Tensor("AdamWeightDecay/gradients/StatefulPartitionedCall:2", shape=(None,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


281/281 [==============================] - 23s 82ms/step - loss: 0.1479 - accuracy: 0.9558 - val_loss: 0.2378 - val_accuracy: 0.9304


In [ ]:
classifier_model.summary()

Model: "prediction"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_4 (KerasLayer)   multiple                  109482241 
_________________________________________________________________
dropout_4 (Dropout)          multiple                  0         
_________________________________________________________________
dense_4 (Dense)              multiple                  1538      
Total params: 109,483,779
Trainable params: 109,483,778
Non-trainable params: 1
_________________________________________________________________


In [ ]:
len(classifier_model.layers)

3

In [ ]:
bert_layer = classifier_model.layers[0]

In [ ]:
dense_layer_weights = classifier_model.layers[2].get_weights()

In [ ]:
bert_layer_weights = bert_layer.get_weights()

In [ ]:
type(bert_layer_weights[0])

numpy.ndarray

In [ ]:
with open(drive_path + 'models/bert_weights0.pkl', 'wb') as f:
    pickle.dump(bert_layer_weights, f)

In [97]:
with open(drive_path + 'models/dense_weights0.pkl', 'wb') as f:
    pickle.dump(dense_layer_weights, f)

In [ ]:
numpyarr = bert_layer.weights[0].numpy()

In [95]:
dense_layer_weights

[array([[-0.0387072 , -0.06983679],
        [ 0.04244252,  0.06597066],
        [ 0.00607397,  0.07949672],
        ...,
        [ 0.0658686 ,  0.02758688],
        [-0.0628758 ,  0.02941896],
        [-0.04358365, -0.08491343]], dtype=float32),
 array([-4.8145408e-05,  4.8145550e-05], dtype=float32)]

In [ ]:
import tensorflow as tf

In [ ]:
drive_path = '/content/drive/MyDrive/amazon_reviews/'

In [18]:
test_strings = ['I love this camera so much, it is fantastic','this thing is awful I dont like it']

In [ ]:
test_model.signatures['serving_default']

<ConcreteFunction signature_wrapper(*, text) at 0x7FDBA50B1090>

In [19]:
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_preprocess = hub.load(tfhub_handle_preprocess)

In [20]:
tok = bert_preprocess.tokenize(tf.constant(test_strings))

In [21]:
text_preprocessed = bert_preprocess.bert_pack_inputs([tok], tf.constant(128))

In [17]:
oob_model = build_classifier_model(2)

In [22]:
logits = oob_model(text_preprocessed)
tf.keras.activations.softmax(logits)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.5236266 , 0.47637337],
       [0.6675831 , 0.33241695]], dtype=float32)>

In [25]:
with open(drive_path + 'models/bert_weights0.pkl', 'rb') as f:
    bert_weights = pickle.load(f)

In [26]:
with open(drive_path + 'models/dense_weights0.pkl', 'rb') as f:
    dense_weights = pickle.load(f)

In [27]:
oob_model.layers[0].set_weights(bert_weights)

In [28]:
oob_model.layers[2].set_weights(dense_weights)

In [29]:
logits = oob_model(text_preprocessed)
tf.keras.activations.softmax(logits)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.00315674, 0.9968432 ],
       [0.9925835 , 0.0074165 ]], dtype=float32)>

In [ ]:
predict_dataset = eval_dataset.map(lambda image, label: image)
for batch in predict_dataset.take(1):
print(inference_func(batch)) 

In [ ]:
with tf.device('/job:localhost'):
    load_options = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
   # test_model = tf.saved_model.load(drive_path + 'models/amzn_model3', options=load_options)
    ds = tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(test_strings, tf.string, name='text')
        )
    )

  #  model_inputs = [[record[ft]] for ft in sentence_features]
 #   spec = tf.TensorSpec(ds)

    logits = test_model(ds2)
    print(tf.keras.activations.softmax(logits))

In [ ]:
ds2 = tf.data.Dataset.from_tensors(tf.cast(test_strings, tf.string, name='text'))

In [ ]:
with tf.device('/job:localhost'):
    for thing in ds2.take(1):
        test_model.signatures['serving_default'](text=thing)


ValueError: ignored

In [ ]:
text_preprocessed

{'input_mask': <tf.Tensor: shape=(1, 20), dtype=int32, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_type_ids': <tf.Tensor: shape=(1, 20), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 20), dtype=int32, numpy=
 array([[  101,  1045,  2293,  2023,  4950,  2061,  2172,  1010,  2009,
          2003, 10392,   102,     0,     0,     0,     0,     0,     0,
             0,     0]], dtype=int32)>}

_NotOkStatusException: ignored

In [ ]:
np.exp(logits)

array([[ 0.1146284, 14.303767 ]], dtype=float32)

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 4.8 MB/s 
     |████████████████████████████████| 636 kB 65.7 MB/s 
     |████████████████████████████████| 895 kB 41.7 MB/s 
     |████████████████████████████████| 3.3 MB 57.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from collections import Counter
import csv
from transformers import pipeline
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [ ]:
drive_path = '/content/drive/MyDrive/amazon_reviews/data/'

In [ ]:
train_df_full = pd.read_csv(drive_path + 'train.csv')
test_df_full =pd.read_csv(drive_path + 'test.csv')

In [ ]:
#classifier = pipeline('sentiment-analysis')

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification, BertTokenizerFast, BertModel
from transformers import InputExample, InputFeatures


tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [ ]:

def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'text'
LABEL_COLUMN = 'rating'

In [ ]:
train_df = train_df_full[:36000].copy()
test_df = test_df_full[:4000].copy()

In [ ]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train_df, 
                                                                           test_df, 
                                                                           'text', 
                                                                           'rating')

In [ ]:
train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
print("Connecting to TPU")
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print(f"Running on TPU {tpu.master()}")
except ValueError:
    print("Could not connect to TPU")
    tpu = None

if tpu:
    try:
        print("Initializing TPU")
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        tpu = tpu
        print("TPU initialized")
    except _:
        print("Failed to initialize TPU")
else:
    print("Unable to initialize TPU")

In [ ]:
print(f"GPUs Available: {len(tf.config.experimental.list_physical_devices('GPU'))}")

GPUs Available: 1


In [ ]:
strategy = tf.distribute.get_strategy()
AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync

In [ ]:
#with strategy.scope():

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5, epsilon=1e-08, clipnorm=1.0), 
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
            metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.fit(train_data, epochs=3, validation_data=validation_data, verbose=1)

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

KeyboardInterrupt: ignored

In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [ ]:
 model.layers[2].output_shape

AttributeError: ignored

In [ ]:
layer.weights

[<tf.Variable 'tf_bert_for_sequence_classification/classifier/kernel:0' shape=(768, 2) dtype=float32, numpy=
 array([[-0.00231929, -0.04074664],
        [-0.01974208,  0.00013093],
        [-0.02377999, -0.00883017],
        ...,
        [ 0.00094265,  0.02442565],
        [ 0.00546543, -0.01884186],
        [ 0.00636708, -0.01589933]], dtype=float32)>,
 <tf.Variable 'tf_bert_for_sequence_classification/classifier/bias:0' shape=(2,) dtype=float32, numpy=array([-0.00017574,  0.00017574], dtype=float32)>]

In [ ]:
import gc
import os
import random
import transformers
import warnings

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K


from pathlib import Path
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, KFold
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from transformers import AutoTokenizer, TFAutoModel

print(f"TensorFlow version: {tf.__version__}")
print(f"Transformers version: {transformers.__version__}")

warnings.filterwarnings("ignore")

TensorFlow version: 2.5.0
Transformers version: 4.9.1


In [ ]:
## defining configuration
class Configuration():
    """
    All configuration for running an experiment
    """
    def __init__(
        self,
        model_name,
        max_length = 64,
        padding = True,
        batch_size = 128,
        epochs = 5,
        learning_rate = 1e-5,
        metrics = ["sparse_categorical_accuracy"],
        verbose = 1,
        train_splits = 5,
        accelerator = "TPU",
        myluckynumber = 13
    ):
        # seed and accelerator
        self.SEED = myluckynumber
        self.ACCELERATOR = accelerator

        # paths
        drive_path = '/content/drive/MyDrive/amazon_reviews/data/'
        self.PATH_TRAIN = Path(drive_path + "train.csv")
        self.PATH_TEST  = Path(drive_path + "test.csv")

        # splits
        self.TRAIN_SPLITS = train_splits


        # model configuration
        self.MODEL_NAME = model_name
        self.TOKENIZER = AutoTokenizer.from_pretrained(self.MODEL_NAME)
        
        # model hyperparameters
        self.MAX_LENGTH = max_length
        self.PAD_TO_MAX_LENGTH = padding
        self.BATCH_SIZE = batch_size
        self.EPOCHS = epochs
        self.LEARNING_RATE = learning_rate
        self.METRICS = metrics
        self.VERBOSE = verbose
        
        # initializing accelerator
        self.initialize_accelerator()

    def initialize_accelerator(self):
        """
        Initializing accelerator
        """
        # checking TPU first
        if self.ACCELERATOR == "TPU":
            print("Connecting to TPU")
            try:
                tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
                print(f"Running on TPU {tpu.master()}")
            except ValueError:
                print("Could not connect to TPU")
                tpu = None

            if tpu:
                try:
                    print("Initializing TPU")
                    tf.config.experimental_connect_to_cluster(tpu)
                    tf.tpu.experimental.initialize_tpu_system(tpu)
                    self.strategy = tf.distribute.experimental.TPUStrategy(tpu)
                    self.tpu = tpu
                    print("TPU initialized")
                except _:
                    print("Failed to initialize TPU")
            else:
                print("Unable to initialize TPU")
                self.ACCELERATOR = "GPU"

        # default for CPU and GPU
        if self.ACCELERATOR != "TPU":
            print("Using default strategy for CPU and single GPU")
            self.strategy = tf.distribute.get_strategy()

        # checking GPUs
        if self.ACCELERATOR == "GPU":
            print(f"GPUs Available: {len(tf.config.experimental.list_physical_devices('GPU'))}")

        # defining replicas
        self.AUTO = tf.data.experimental.AUTOTUNE
        self.REPLICAS = self.strategy.num_replicas_in_sync
        print(f"REPLICAS: {self.REPLICAS}")

In [ ]:
def encode_text(df, tokenizer, max_len, padding):
    """
    Preprocessing textual data into encoded tokens.
    """
    text = df[['text']].values.tolist()

    # encoding text using tokenizer of the model
    text_encoded = tokenizer.batch_encode_plus(
        text,
        pad_to_max_length = padding,
        max_length = max_len
    )

    return text_encoded



In [ ]:

def get_tf_dataset(X, y, auto, labelled = True, repeat = False, shuffle = False, batch_size = 128):
    """
    Creating tf.data.Dataset for TPU.
    """
    if labelled:
        ds = (tf.data.Dataset.from_tensor_slices((X["input_ids"], y)))
    else:
        ds = (tf.data.Dataset.from_tensor_slices(X["input_ids"]))

    if repeat:
        ds = ds.repeat()

    if shuffle:
        ds = ds.shuffle(2048)

    ds = ds.batch(batch_size)
    ds = ds.prefetch(auto)

    return ds

In [ ]:
## building model
def build_model(model_name, max_len, learning_rate, metrics):
    """
    Building the Deep Learning architecture
    """
    # defining encoded inputs
    input_ids = Input(shape = (max_len,), dtype = tf.int32, name = "input_ids")
    
    # defining transformer model embeddings
    transformer_model = TFAutoModel.from_pretrained(model_name)
    transformer_embeddings = transformer_model(input_ids)[0]

    # defining output layer
    output_values = Dense(1, activation = "sigmoid")(transformer_embeddings[:, 0, :])

    # defining model
    model = Model(inputs = input_ids, outputs = output_values)
    opt = Adam(learning_rate = learning_rate)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
    metrics = metrics

    model.compile(optimizer = opt, loss = loss, metrics = metrics)

    return model

In [ ]:
def run_model(config):
    """
    Running the model
    """
    ## reading data
    df_train = pd.read_csv(config.PATH_TRAIN)
    df_test = pd.read_csv(config.PATH_TEST)
    
    # adding column for stratified splitting
 
    # stratified K-fold on language and label
    kf = KFold(n_splits = config.TRAIN_SPLITS, shuffle = False, random_state = config.SEED)

    # initializing predictions
    preds_oof = np.zeros((df_train.shape[0], 1))
    preds_test = np.zeros((df_test.shape[0], 1))
    acc_oof = []

    # iterating over folds
    for (fold, (train_index, valid_index)) in enumerate(kf.split(df_train)):
        # initializing TPU
        if config.ACCELERATOR == "TPU":
            if config.tpu:
                config.initialize_accelerator()

        # building model
        K.clear_session()
        with config.strategy.scope():
            model = build_model(config.MODEL_NAME, config.MAX_LENGTH, config.LEARNING_RATE, config.METRICS)
            if fold == 0:
                print(model.summary())

        print("\n")
        print("#" * 19)
        print(f"##### Fold: {fold + 1} #####")
        print("#" * 19)

        # splitting data into training and validation
        X_train = df_train.iloc[train_index]
        X_valid = df_train.iloc[valid_index]

        y_train = X_train.rating.values
        y_valid = X_valid.rating.values

        print("\nTokenizing")

          # encoding text data using tokenizer
        X_train_encoded = encode_text(df = X_train, tokenizer = config.TOKENIZER, max_len = config.MAX_LENGTH, padding = config.PAD_TO_MAX_LENGTH)
        X_valid_encoded = encode_text(df = X_valid, tokenizer = config.TOKENIZER, max_len = config.MAX_LENGTH, padding = config.PAD_TO_MAX_LENGTH)

        # creating TF Dataset
        ds_train = get_tf_dataset(X_train_encoded, y_train, config.AUTO, repeat = True, shuffle = True, batch_size = config.BATCH_SIZE * config.REPLICAS)
        ds_valid = get_tf_dataset(X_valid_encoded, y_valid, config.AUTO, batch_size = config.BATCH_SIZE * config.REPLICAS * 4)

        n_train = X_train.shape[0]

        if fold == 0:
            X_test_encoded = encode_text(df = df_test, tokenizer = config.TOKENIZER, max_len = config.MAX_LENGTH, padding = config.PAD_TO_MAX_LENGTH)

        # saving model at best accuracy epoch
        sv = tf.keras.callbacks.ModelCheckpoint(
            "model.h5",
            monitor = "val_sparse_categorical_accuracy",
            verbose = 0,
            save_best_only = True,
            save_weights_only = True,
            mode = "max",
            save_freq = "epoch"
        )

        print("\nTraining")

        # training model
        model_history = model.fit(
            ds_train,
            epochs = config.EPOCHS,
            callbacks = [sv],
            steps_per_epoch = n_train / config.BATCH_SIZE // config.REPLICAS,
            validation_data = ds_valid,
            verbose = config.VERBOSE
        )

        print("\nValidating")

        # scoring validation data
        model.load_weights("model.h5")
        ds_valid = get_tf_dataset(X_valid_encoded, -1, config.AUTO, labelled = False, batch_size = config.BATCH_SIZE * config.REPLICAS * 4)

        preds_valid = model.predict(ds_valid, verbose = config.VERBOSE)
        acc = accuracy_score(y_valid, np.argmax(preds_valid, axis = 1))

        preds_oof[valid_index] = preds_valid
        acc_oof.append(acc)

        print("\nInferencing")

        # scoring test data
        ds_test = get_tf_dataset(X_test_encoded, -1, config.AUTO, labelled = False, batch_size = config.BATCH_SIZE * config.REPLICAS * 4)
        preds_test += model.predict(ds_test, verbose = config.VERBOSE) / config.TRAIN_SPLITS

        print(f"\nFold {fold + 1} Accuracy: {round(acc, 4)}\n")

        g = gc.collect()

    # overall CV score and standard deviation
    print(f"\nCV Mean Accuracy: {round(np.mean(acc_oof), 4)}")
    print(f"CV StdDev Accuracy: {round(np.std(acc_oof), 4)}\n")

    return preds_oof, preds_test

In [ ]:
# Model: Bert Base Uncased
config = Configuration("bert-base-uncased", max_length = 512, batch_size = 128, epochs = 2, train_splits = 3)
preds_train_2, preds_test_2 = run_model(config)

PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
Connecting to TPU
Running on TPU grpc://10.62.127.226:8470
Initializing TPU


INFO:tensorflow:Initializing the TPU system: grpc://10.62.127.226:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.62.127.226:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

TPU initialized
REPLICAS: 8


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 512)]             0         
_________________________________________________________________
tf_bert_model (TFBertModel)  TFBaseModelOutputWithPool 109482240 
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 769       
Total params: 109,483,009
Trainable params: 109,483,009
Non-trainable params: 0
_________________________________________________________________
None


###################
##### Fold: 1 #####
###################

Tokenizing


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


TypeError: ignored